# TODO list

In [ ]:
# todo: 在coco上得到各改进模型的预训练模型，然后在lard上训练
# todo: 设计改进yolov8结构

# 改进Yolo模型结构测试

In [1]:
# 注意力模块测试
from ultralytics import YOLO

for att in ['se', 'cbam', 'eca', 'ese', 'gam', 'sa']:
    model = YOLO(f'cfg/ultra/yaml/yolov8n_{att}.yaml').load('./weights/yolov8n.pt')



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

# Fiftyone 
## 查看已有数据集

In [2]:
import fiftyone as fo
all_datasets = fo.list_datasets()
print(all_datasets)

['lard_test_real_edge', 'lard_test_real_nominal', 'lard_test_synth', 'lard_train', 'lard_train_all', 'lard_val']


## 打开web界面查看结果

In [4]:
import fiftyone as fo
dataset = fo.load_dataset("lard_test_synth")
print(dataset.get_field_schema().keys())
session = fo.launch_app(dataset, height=700, auto=False)
session.show()

odict_keys(['id', 'filepath', 'tags', 'metadata', 'ground_truth_detections', 'predictions_yolov8n', 'predictions_yolov8n_aug', 'predictions_yolov8n-p2', 'predictions_faster_rcnn'])
Session launched. Run `session.show()` to open the App in a cell output.


In [5]:
session.close()

## 删除全部预测

In [8]:
from src.utils.database import delete_all_predictions
    
for name in all_datasets:
    dataset = fo.load_dataset(name)
    delete_all_predictions(dataset)


删除字段: []
删除字段: []
删除字段: []
删除字段: []
删除字段: []
删除字段: []


: 

## 评估各数据上各模型性能

In [ ]:
from src.utils.database import eval_detections_in_database

all_results = eval_detections_in_database('yolov8n-p2_train_val')

# todo: 写一个函数，自动处理fiftyone内部 各数据集/各模型 进行评估

In [ ]:
for datatype, results in all_results.items():
    plot_pr = results.plot_pr_curves()
    display(plot_pr)

In [ ]:
for datatype, results in all_results.items():
    plot_mat = results.plot_confusion_matrix()
    display(plot_mat)


# 结果文件分析

In [28]:
# todo 读取results/tables/metrics下各文件 做对比分析

import os
import glob
import pandas as pd

path = 'results/tables/metrics'
all_csv = [file for file in os.listdir(path) if file.endswith('csv')]

def get_model_name(file):
    return file.split('metrics_')[1].split('.csv')[0]

map5095 = []
map50 = []
for file in all_csv:
    df = pd.read_csv(os.path.join(path, file))

    df1 = df.iloc[0:1, :].copy()
    df1['metrics'] = get_model_name(file)
    map5095.append(df1)

    df2 = df.iloc[1:2, :].copy()
    df2['metrics'] = get_model_name(file)
    map50.append(df2)

map5095 = pd.concat(map5095, axis=0)
map50 = pd.concat(map50, axis=0)
print('map5095')
display(map5095)
print('map50')
display(map50)



map5095


,metrics,test_synth,test_real_nominal,test_real_edge
0,yolov8n,0.755,0.606,0.381
0,yolov8n_aug,0.762,0.603,0.459


map50


,metrics,test_synth,test_real_nominal,test_real_edge
1,yolov8n,0.976,0.854,0.613
1,yolov8n_aug,0.978,0.864,0.737


# 样本划分评价

In [2]:
import os
import pandas as pd

ROOT_DATA = os.environ.get('LARD_DATA_ROOT_PATH')
ROOT_PROJECT = os.environ.get('LARD_PROJECT_ROOT_PATH')

print(ROOT_DATA)
print(ROOT_PROJECT)

/fileonssd/lard-dataset
/home/yeli/workspace/lard/lard-detection


In [6]:
import json
from pycocotools.coco import COCO
from src.utils.metric import coco_metrics_name
from src.utils.metric import cal_coco_metrics


def cal_coco_metrics_grouped(type, path_info, path_anno, path_pred):

    df = pd.read_csv(path_info, sep=';')

    if type == 'synth':
        bins = [0, 1, 2, 3]
        key_col = 'along_track_distance'
        index_name = 'Along track distance'
    elif type == 'real':
        bins = [0, 10, 20, 40, 80, 160]
        key_col = 'time_to_landing'
        index_name = 'Time to landing'
    else:
        print('Error')
        
    coco_true = COCO(annotation_file=path_anno)

    # 使用pd.cut()函数对数据框进行分组
    df['group'] = pd.cut(df[key_col], bins=bins, labels=False)
    # 分组
    df_group = df.groupby('group')

    # 各组metrics
    all_metrics = {}

    for group_id in range(len(bins)-1):

        # 获取本足图像文件名列表
        file_list = df_group.get_group(group_id).image.apply(lambda x: os.path.basename(x)).to_list()

        temp_folder = './grouped_json'
        os.makedirs(temp_folder, exist_ok=True)

        # 取真值
        # 筛选图像
        imgs_to_keep = []
        anns_to_keep = []

        for img in coco_true.imgs.values():
            if img['file_name'] in file_list:
                imgs_to_keep.append(img)
                ann_ids = coco_true.getAnnIds(imgIds=img['id'])
                anns_to_keep.extend(coco_true.loadAnns(ann_ids))

        # 构建新的JSON数据
        group_instances_data = {
            "images": imgs_to_keep,
            "annotations": anns_to_keep,
            "categories": coco_true.loadCats(coco_true.getCatIds())
        }

        # 保存为分组后的JSON文件
        if type == 'synth':
            group_instances_file = f'{temp_folder}/instances_test_synth_dist{group_id}.json'
        elif type == 'real':
            group_instances_file = f'{temp_folder}/instances_test_synth_time{group_id}.json'
        else:
            print('Error')
        with open(group_instances_file, 'w') as f:
            json.dump(group_instances_data, f, indent=4)

        # 取预测
        # GT对应的图像id
        image_id_to_keep = [ann['image_id'] for ann in anns_to_keep]
        
        with open(path_pred, 'r') as file:
            data = json.load(file)

        # 构建新的JSON数据
        group_predictions_data = [pred for pred in data if pred['image_id'] in image_id_to_keep]

        # 保存为分组后的JSON文件
        group_predictions_file = f'{temp_folder}/predictions_test_synth_dist{group_id}.json'
        with open(group_predictions_file, 'w') as f:
            json.dump(group_predictions_data, f, indent=4)

        # 算误差
        coco_evaluator = cal_coco_metrics(group_instances_file, group_predictions_file)

        # 构建df
        all_metrics[f"{bins[group_id]} to {bins[group_id+1]} nm"] = coco_evaluator.stats

    all_metrics = pd.DataFrame(all_metrics, index=coco_metrics_name)
    all_metrics.index.name = index_name
    display(all_metrics.round(3))

In [7]:
# Synth
path_synth = f'{ROOT_DATA}/LARD_dataset_decompressed/LARD_test/LARD_test_synth/LARD_test_synth.csv'
path_anno = 'datasets/lard/annotations/instances_test_synth.json'
path_pred = 'runs/mmdetection/faster_rcnn/test/coco_detection/prediction_test_synth.bbox.json'
cal_coco_metrics_grouped('synth', path_synth, path_anno, path_pred)

# Real nominal
path_real_nominal = f'{ROOT_DATA}/LARD_dataset_decompressed/LARD_test/LARD_test_real/LARD_test_real_nominal/LARD_test_real_nominal.csv'
path_anno = 'datasets/lard/annotations/instances_test_real_nominal.json'
path_pred = 'runs/mmdetection/faster_rcnn/test/coco_detection/prediction_test_real_nominal.bbox.json'
cal_coco_metrics_grouped('real', path_real_nominal, path_anno, path_pred)

# Edge cases
path_real_edge = f'{ROOT_DATA}/LARD_dataset_decompressed/LARD_test/LARD_test_real/LARD_test_real_edge_cases/LARD_test_real_edge_cases.csv'
path_anno = 'datasets/lard/annotations/instances_test_real_edge.json'
path_pred = 'runs/mmdetection/faster_rcnn/test/coco_detection/prediction_test_real_edge.bbox.json'
cal_coco_metrics_grouped('real', path_real_edge, path_anno, path_pred)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.36s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.820
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.990
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.956
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.750
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.821
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.865
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | max

,0 to 1 nm,1 to 2 nm,2 to 3 nm
Along track distance,,,
"AP (IoU=0.50:0.95, area=all, maxDets=100)",0.820,0.672,0.568
"AP (IoU=0.50, area=all, maxDets=100)",0.990,0.965,0.885
"AP (IoU=0.75, area=all, maxDets=100)",0.956,0.812,0.672
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-1.000,-1.000,-1.000
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.750,0.663,0.568
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.821,0.680,0.552
"AR (IoU=0.50:0.95, area=all, maxDets=1)",0.865,0.726,0.610
"AR (IoU=0.50:0.95, area=all, maxDets=10)",0.866,0.727,0.610
"AR (IoU=0.50:0.95, area=all, maxDets=100)",0.866,0.727,0.610


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.14s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.806
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.983
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.948
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.746
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.810
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.834
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | max

,0 to 10 nm,10 to 20 nm,20 to 40 nm,40 to 80 nm,80 to 160 nm
Time to landing,,,,,
"AP (IoU=0.50:0.95, area=all, maxDets=100)",0.806,0.697,0.405,0.164,0.016
"AP (IoU=0.50, area=all, maxDets=100)",0.983,0.905,0.621,0.284,0.024
"AP (IoU=0.75, area=all, maxDets=100)",0.948,0.821,0.492,0.178,0.022
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-1.000,0.000,0.251,0.142,0.029
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.746,0.615,0.464,0.257,0.073
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.810,0.761,0.641,0.404,-1.000
"AR (IoU=0.50:0.95, area=all, maxDets=1)",0.834,0.730,0.464,0.213,0.034
"AR (IoU=0.50:0.95, area=all, maxDets=10)",0.839,0.741,0.492,0.234,0.034
"AR (IoU=0.50:0.95, area=all, maxDets=100)",0.839,0.741,0.492,0.234,0.034


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.467
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.702
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.506
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.370
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.476
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.502
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | max

,0 to 10 nm,10 to 20 nm,20 to 40 nm,40 to 80 nm,80 to 160 nm
Time to landing,,,,,
"AP (IoU=0.50:0.95, area=all, maxDets=100)",0.467,0.246,0.138,0.029,0.0
"AP (IoU=0.50, area=all, maxDets=100)",0.702,0.400,0.222,0.057,0.0
"AP (IoU=0.75, area=all, maxDets=100)",0.506,0.258,0.159,0.032,0.0
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-1.000,0.000,0.089,0.177,0.0
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.370,0.292,0.191,0.000,0.0
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.476,0.232,0.000,-1.000,-1.0
"AR (IoU=0.50:0.95, area=all, maxDets=1)",0.502,0.269,0.178,0.097,0.0
"AR (IoU=0.50:0.95, area=all, maxDets=10)",0.531,0.279,0.178,0.097,0.0
"AR (IoU=0.50:0.95, area=all, maxDets=100)",0.531,0.279,0.178,0.097,0.0
